In [1]:
# import some necessary python modules 
import numpy as np
import pandas as pd
import random 
import os
import glob 
import tqdm
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
#! pip install pyarrow
plt.style.use('classic')
import matplotlib as mpl
import scipy

In [ ]:
print("tqdm version:", tqdm.__version__)
print("matplotlib version:", mpl.__version__)
print("scipy version:", scipy.__version__)

In [2]:
# create a folder 
if not os.path.exists('Imagefolder_100'):
    os.mkdir('Imagefolder_100')
if not os.path.exists('Heatmap_100'):
    os.mkdir('Heatmap_100')

In [3]:
# define the frame dimensions
X_frames = 128
Y_frames = 128

# measure typical number density
Packing_fraction  = 0.7
total_Area  = X_frames*Y_frames
particle_radius =  22.5/1300*64

N = total_Area/(np.pi*particle_radius**2)

In [4]:
# define a function that will generate the images for you

def image_generator(outpath_path, image_size, N_particles, PtoB_ratio=0.1, packing_frac = 0.5):
    '''
    Generates N images of a specified size in .jpg format
    Params:
        output_path: 
    Outputs:
        
    '''
    # define the frame dimensions
    X_frames = image_size[0]
    Y_frames = image_size[1]

    # for the grids that will be later made
    xedges =  np.linspace(-X_frames/2, X_frames/2, X_frames+1)
    yedges =  np.linspace(-Y_frames/2, Y_frames/2, Y_frames+1)

    # measure typical number density
    Packing_fraction  = packing_frac

    # define the particle to box ratio
    PtoB_ratio = PtoB_ratio

    # generate a number between these values
    num_circles = N_particles

    # create a smapling list a bit smaller than the image dimensions
    length_rand = int(np.floor(X_frames/10)*10)
    
    random_list_ = sorted(np.unique(np.array(np.linspace(-length_rand/2, length_rand/2, length_rand), dtype = int)))

    # pick for the list
    Xx = random.sample(random_list_, num_circles)
    Yy = random.sample(random_list_, num_circles)

    # scale the coorinates to the correct size
    X = np.array(Xx)/X_frames*1000/10000
    Y = np.array(Yy)/Y_frames*1000/10000
    scale_ = 4

    # develop the image with consistent dimensions
    fig = plt.figure(figsize = (scale_, scale_))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.axis('equal')
    ax.set_axis_off()
    fig.add_axes(ax)
    plt.scatter(X,Y, c = 'k', s = (PtoB_ratio*scale_*70.7)**2)
    plt.xlim([-0.05, 0.05])
    plt.ylim([-0.05, 0.05])
    plt.savefig(outpath_path+'.png', dpi=X_frames/scale_)
    plt.close()

    # check for duplicates
    # duplicates can cause problems with the output
    data_check = pd.DataFrame({'X':Xx, 'Y':Yy})
    data_check.sort_values(by = ['X'])
    vall1 = data_check.groupby('X')['Y'].count()
    vall2 = data_check.groupby('X')['Y'].nunique()
    dupli = np.sum(vall1.values- vall2.values)

    # now create the simulated response
    H, xedges, yedges = np.histogram2d(Yy, Xx, bins=(xedges, yedges))

    return Xx,Yy, dupli, np.flipud(H)

In [5]:
# create a for loop for geerating the images
def gen_path(folder, N_images):
    '''
    This function is used to create path list for the images
    '''
    path_list = [(folder + '\\' +str(n).zfill(6)) for n in range(N_images)]
    return path_list   

In [7]:
# create a bunch of images t  

folder =r'Imagefolder_100'
N_images = 10

path_list = gen_path(folder, N_images)

H_list = []
for i in tqdm.tqdm(range(N_images)) :
    X_, Y_, d_, H = image_generator(path_list[i], [128, 128],100, PtoB_ratio=17/512, packing_frac = 0.5)
    H_list.append(H.flatten())

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 23.57it/s]


In [19]:
# sav the y values in a parquet format 
H_val = pd.DataFrame()
H_val['H'] = H_list
H_val.to_parquet('H_values_100.parquet')

In [22]:
# create heatmaps
for i in tqdm(range(len(H_val))):
    sigma = 0.7  # Spread of the Gaussian

    binary_mask =  H_val['H'].iloc[i].reshape(128, 128) # Should be shape (128, 128) with 1s at centers
    heatmap = gaussian_filter(binary_mask.astype(np.float32), sigma=sigma)
    heatmap = np.clip(heatmap, 0, 1)
    

    np.save(path_list[i]+'.npy', heatmap)

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 338.50it/s]
